In [ ]:
import optuna
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def load_study(study_name):
    journal_name = f'{study_name}.log'
    study_storage = optuna.storages.JournalStorage(
        optuna.storages.JournalFileStorage(journal_name)) 
    study = optuna.create_study(study_name=study_name,
                                storage=study_storage,
                                direction='minimize',
                                load_if_exists=True)
    return study

In [ ]:
trials_df = []
for study_name in ['criteo_bins', 'criteo_splines_0', 'criteo_splines_3']:
    study = load_study(study_name)
    print(f'Study {study_name} has {len(study.trials)} trials')

    trial_data = [trial.params | trial.user_attrs | dict(val_loss=trial.value, study_name=study_name)
                  for trial in [study.best_trial]]
    trial_data_df = pd.DataFrame.from_records(trial_data)
    trials_df.append(trial_data_df)

trials_df = pd.concat(trials_df, axis=0)
trials_df

In [ ]:
pivoted = trials_df[['study_name', 'test_loss']].pivot(columns=['study_name'], values=['test_loss'])
pivoted

In [ ]:
lift_pct = (100 * (1 - pivoted.iloc[:, 1:] / pivoted.iloc[:, 0].values.item()))
lift_pct